# Seq2Seq 기반 챗봇 만들기

> 1. 데이터 만들기

In [ ]:
# KoNLPy 설치
import os

# KoNLPy 라이브러리 설치
!pip install konlpy

# jdk, JPype1-py3 설치
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip3 install JPype1-py3

# automake 설치
os.chdir('/tmp')
!curl -LO http://ftpmirror.gnu.org/automake/automake-1.11.tar.gz
!tar -zxvf automake-1.11.tar.gz
os.chdir('/tmp/automake-1.11')
!./configure
!make
!make install

# automake 설치 오류 시
os.chdir('/tmp/') 
!wget -O m4-1.4.9.tar.gz http://ftp.gnu.org/gnu/m4/m4-1.4.9.tar.gz
!tar -zvxf m4-1.4.9.tar.gz
os.chdir('/tmp/m4-1.4.9')
!./configure
!make
!make install
os.chdir('/tmp')
!curl -OL http://ftpmirror.gnu.org/autoconf/autoconf-2.69.tar.gz
!tar xzf autoconf-2.69.tar.gz
os.chdir('/tmp/autoconf-2.69')
!./configure --prefix=/usr/local
!make
!make install
!export PATH=/usr/local/bin

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 9.7MB/s 
     |████████████████████████████████| 3.8MB 45.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u252-b09-1~18.04_amd64.deb  404  Not Found [IP: 91.189.88.142 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jdk-headless_8u252-b09-1~18.04_amd64.deb  404  Not Found [IP: 91.189.88.142 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?
     |████████████████████████████████| 92kB 3.2MB/s 
  Created wheel for JPype1-py3: filename=JPype1_py3-0.5.5.4-cp36-cp36m-linux_x86_64.whl size=2

In [ ]:
# 모듈 불러오기
from konlpy.tag import Okt
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import pickle

In [ ]:
# 경로 설정
root_path = "/content/drive/My Drive/멀티캠퍼스/[혁신성장] 인공지능 자연어처리 기반/[강의]/조성현 강사님"
data_path = f"{root_path}/dataset"
chatbot_path = f"{root_path}/Seq2Seq-Chatbot"

In [ ]:
# 파일 이름 설정
DATA_PATH = f"{data_path}/6-1.ChatBotData.csv"
VOCABULARY_PATH = f"{data_path}/6-1.vocabulary.voc"

## 파라미터 설정

 챗봇을 만들 때는 형태소로 대답하면 안 되니까 `False` 파라미터로 설정한다.

In [ ]:
# 데이터 로드를 위한 파라미터 설정
TOKENIZE_AS_MORPH = False # 형태소로 대답하면 안 됨.
ENC_INPUT = 0 # encoder 입력
DEC_INPUT = 1 # decoder 입력
DEC_LABEL = 2 # decoder 출력
MAX_LENGTH = int(input('문장 최대 길이 설정: '))

문장 최대 길이 설정: 10


In [ ]:
# 특별 토큰
FILTERS = "([~.,!?\"':;)(])"
PAD = "<PADDING>"
STD = "<START>"
END = "<END>"
UNK = "<UNKNOWN>"

MARKER = [PAD, STD, END, UNK]
CHANGE_FILTER = re.compile(FILTERS)

## 데이터 로드

In [ ]:
# 데이터 로드 함수
def load_data(path):
    data = pd.read_csv(path, header=0)
    question, answer = list(data['Q']), list(data['A'])
    train_input, test_input, train_label, test_label = \
        train_test_split(question, answer, test_size=0.1, random_state=42)
    return train_input, train_label, test_input, test_label

In [ ]:
# 데이터 로드 후 확인
train_input, train_label, test_input, test_label = load_data(DATA_PATH)
print(f"Train: {len(train_input)}, {len(train_label)}")
print(f"Sample Train data:\n      Input: {train_input[0]}, Label: {train_label[0]}")
print()
print(f"Test: {len(test_input)}, {len(test_label)}")
print(f"Sample Test data:\n      Input: {test_input[0]}, Label: {test_label[0]}")

Train: 10640, 10640
Sample Train data:
      Input: 나한테 질리면 어쩌지 걱정돼, Label: 당신의 겉모습이 아닌 진정한 내면의 모습을 보여주세요.

Test: 1183, 1183
Sample Test data:
      Input: 죽을거 같네, Label: 나쁜 생각 하지 마세요.


## 형태소 분석(적용 x)

In [ ]:
# 형태소 분석 함수
def morphlize_data(tagger, data):
    tagger = tagger()
    result = []
    for sent in tqdm(data):
        morph = " ".join(tagger.morphs(sent.replace(" ", "")))
        result.append(morph)
        
    return result

## 인코더, 디코더용 데이터 만들기
- 사전: 원래는 빈도별로 해야 하는데 그렇게 안 함. 가나다 순.
- 전처리
    - 형태소 분석
    - OOV 처리

In [ ]:
# 토크나이징
def tokenize_data(data):
    words = []
    for sentence in data:
        sentence = re.sub(CHANGE_FILTER, "", sentence)
        for word in sentence.split():
            words.append(word)
            
    return [word for word in words if word]

In [ ]:
# 사전 파일 만들기
def make_vocabulary(path, tokenize_as_morph=TOKENIZE_AS_MORPH):
    data = pd.read_csv(path, encoding='utf-8')
    question, answer = list(data['Q']), list(data['A'])
    
    # 형태소 분석
    if TOKENIZE_AS_MORPH:  
        question = morphlize_data(Okt, question)
        answer = morphlize_data(Okt, answer)
    
    # 어휘집 생성: 앞에 특별 토큰 추가
    sentences = []
    sentences.extend(question)
    sentences.extend(answer)
    words = tokenize_data(sentences)
    words = list(set(words)) # 중복 단어 제거
    words[:0] = MARKER

    word2idx = {word: idx for idx, word in enumerate(words)}
    idx2word = {idx: word for idx, word in enumerate(words)}
    
    return word2idx, idx2word

In [ ]:
def preprocess_data(data, dictionary, dataType, tokenize_as_morph=TOKENIZE_AS_MORPH, filter=CHANGE_FILTER, maxlen=MAX_LENGTH):
    # 형태소 토크나이징 여부
    if tokenize_as_morph:
        data = morphlize_data(data)
    
    sequences_input_index = []
    for sequence in data:
        sequence = re.sub(filter, "", sequence) # 구둣점 제거

        # 디코더 입력: <START>로 시작
        if dataType == DEC_INPUT: 
            sequence_index = [dictionary[STD]]
        else:
            sequence_index = []

        # OOV: <unknown>
        for word in sequence.split():
            if dictionary.get(word) is not None:
                sequence_index.append(dictionary[word])
            else:
                sequence_index.append(dictionary[UNK])

            # 문장 단어 수 제한
            if len(sequence_index) >= maxlen:
                break
        
        # 디코더 출력: <END>로 끝
        if dataType == DEC_LABEL:
            if len(sequence_index) < maxlen:
                sequence_index.append(dictionary[END])
            else:
                sequence_index[len(sequence_index)-1] = dictionary[END]
        
        # 패딩: maxlen보다 작을 경우
        sequence_index += (maxlen - len(sequence_index))*[dictionary[PAD]]
        sequences_input_index.append(sequence_index)
    
    return np.asarray(sequences_input_index)

In [ ]:
# 사전 생성
word2idx, idx2word = make_vocabulary(DATA_PATH)
print(f"word-index : {list(word2idx.items())[:20]}")
print(f"Vocab Size: {len(word2idx)}")

word-index : [('<PADDING>', 0), ('<START>', 1), ('<END>', 2), ('<UNKNOWN>', 3), ('치료하세요', 4), ('관계란', 5), ('개째', 6), ('끝내려고', 7), ('자유로워질까요', 8), ('며칠', 9), ('안보는', 10), ('방금', 11), ('꿈에나온너', 12), ('힘들어졌어요', 13), ('갑작스러웠나봐요', 14), ('통화를', 15), ('전화하기엔', 16), ('헤어진사람에게', 17), ('힘들게', 18), ('들어가', 19)]
Vocab Size: 20705


In [ ]:
# 학습 데이터 : 인코더 입력, 디코더 입력, 디코더 출력
train_input_E = preprocess_data(train_input, word2idx, ENC_INPUT)
train_input_D = preprocess_data(train_label, word2idx, DEC_INPUT)
train_label_D = preprocess_data(train_label, word2idx, DEC_LABEL)

In [ ]:
# 평가 데이터 : 인코더 입력, 디코더 입력, 디코더 출력
test_input_E = preprocess_data(test_input, word2idx, ENC_INPUT)
test_input_D = preprocess_data(test_label, word2idx, DEC_INPUT)
test_label_D = preprocess_data(test_label, word2idx, DEC_LABEL)

## 결과 저장

In [ ]:
# vocabulary 저장
with open(f"{chatbot_path}/vocabulary.pickle", 'wb') as f:
    pickle.dump([word2idx, idx2word], f, pickle.HIGHEST_PROTOCOL)

# train data 저장
with open(f"{chatbot_path}/train.pickle", 'wb') as f:
    pickle.dump([word2idx, idx2word], f, pickle.HIGHEST_PROTOCOL)

# test data 저장
with open(f"{chatbot_path}/train.pickle", 'wb') as f:
    pickle.dump([word2idx, idx2word], f, pickle.HIGHEST_PROTOCOL)

# ===== 테스트 =====

데이터 원래 생김새

In [ ]:
df = pd.read_csv(DATA_PATH)
display(df)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2
